In [32]:
import os
dataset = '/ssd2/lz/MM2022/dataset/FE108/test/'
sequence = []
try:
    with open(os.path.join(dataset, 'list.txt'), 'r') as f:
        value = f.readlines()
    for seq in value:
        sequence.append(seq.split("\n")[0])
    sequence.sort()
except:    
    sequence =  os.listdir(dataset)
    sequence.sort()
    with open(os.path.join(dataset, "list.txt"), 'w') as f:
        for seq in sequence:
            f.writelines(seq + "\n")

In [31]:

for index,seq in enumerate(sequence):
    visimg = os.path.join(dataset, seq, "event_imgs")
    colorimage = os.path.join(dataset, seq,"img")
    # os.rename(visimg, colorimage) 
    imageslist = os.listdir(colorimage)
    imageslist.sort()
    infraredlist = os.listdir(visimg)
    infraredlist.sort()
    assert len(imageslist) == len(infraredlist)
    imgsuffix = imageslist[0].split(".")[1]
    infraredsuffix = infraredlist[0].split(".")[1] 
    for i in range(len(imageslist)):
        colorname = os.path.join(colorimage, imageslist[i])
        sortname = os.path.join(colorimage, "%08d.%s" % (i+1, imgsuffix))
        os.rename(colorname, sortname)

    for i in range(len(infraredlist)):
        visiblename = os.path.join(visimg, infraredlist[i])
        sortname = os.path.join(visimg, "%08d.%s" % (i+1, infraredsuffix))
        os.rename(visiblename, sortname)

    with open(os.path.join(dataset, seq, "sequence"), 'w') as f:
        line1 = 'channels.color=img/%08d.' + imgsuffix
        line2 = "channels.depth=event_imgs/%08d." + infraredsuffix                                                                                                                                                                          
        line3 = "format=default"                                                                                                                                                                          
        line4 = "fps=30"                                                                                                                                                                            
        line5 = "name={}".format(seq)
        f.writelines(line1 + '\n')
        f.writelines(line2 + '\n')
        f.writelines(line3 + '\n')
        f.writelines(line4 + '\n')
        f.writelines(line5 + '\n')


In [36]:
# import shutil
# for index,seq in enumerate(sequence):
#     visfile = os.path.join(dataset, "../gtfile/", "{}.txt".format(seq))
#     groundtruth = os.path.join(dataset, seq, "groundtruth.txt")
#     shutil.copy2(groundtruth,visfile)


In [24]:
for index,seq in enumerate(sequence):
    visfile = os.path.join(dataset, seq, "groundtruth_rect.txt")
    groundtruth = os.path.join(dataset, seq, "groundtruth.txt")
    os.rename(visfile, groundtruth)
    with open(groundtruth, 'r') as f:
        gt_length = len(f.readlines())
    channel = os.path.join(dataset,seq, 'img')
    frame = len(os.listdir(channel))
    try:
        assert frame == gt_length
    except:
        print("seq: {}".format(seq))

In [40]:
# import shutil
import os
dir = '/ssd2/lz/rgbd_benchmark/all_benchmark/results/transt_prompt'
seq_list = []
with open(os.path.join('/ssd3/lz/MM2022/dataset/depthtrack', "list.txt") ,'r') as f:
    value = f.readlines()
    for val in value:
        seq_list.append(val.split("\n")[0])
seq_list.sort()
for seq in seq_list:
    pre_file = os.path.join(dir,'{}.txt'.format(seq))
    newprefile = os.path.join(dir, "{}_001.txt".format(seq))
    os.rename(pre_file, newprefile)
    time_file = os.path.join(dir, '{}_time.txt'.format(seq))
    newtimefile = os.path.join(dir, '{}_001_time.value'.format(seq))
    os.rename(time_file, newtimefile)

# Event grabcut

In [12]:
import os
import cv2
import numpy as np

event_image = "/ssd2/lz/MM2022/dataset/event/visevent/00141_tank_outdoor2/event_imgs/00000003.png"
color_image = "/ssd2/lz/MM2022/dataset/event/visevent/00141_tank_outdoor2/color/00000003.png"
print(os.path.exists(event_image))
event = cv2.imread(event_image)
rows, cols, channels = event.shape


event2gray = cv2.cvtColor(event, cv2.COLOR_BGR2GRAY)

ret, mask = cv2.threshold(event2gray, 175, 255, cv2.THRESH_BINARY)

# newimage = np.zeros([event.shape[0], event.shape[1], 3], np.uint8)
# newimage[:] = [255,255,255]
newimage= cv2.imread(color_image)
roi = newimage[0:rows, 0:cols]
mask_inv = cv2.bitwise_not(mask)
newimage_bg = cv2.bitwise_and(roi, roi, mask = mask)
img2_fg = cv2.bitwise_and(event, event, mask = mask_inv)

dst = cv2.add(newimage_bg, img2_fg)


# newimage = cv2.add(newimage, )
cv2.imwrite('./white.png',newimage)
cv2.imwrite("./gray3.png",event)
cv2.imwrite("./grab.png", dst)

True


True